## SETUP

<br>Backup C:\spark-2.4.7-bin-hadoop2.7\conf\spark-defaults.conf.template -> spark-defaults.comf.template.bak
<br>Rename & Edit C:\spark-2.4.7-bin-hadoop2.7\conf\spark-defaults.conf.template as "spark-defaults.conf"

```bash
spark-defaults.conf
# is always loaded by spark runtime when it starts....
# here we can mention default parameters needed for spark workers..

# Paste at EOF
spark.jars.packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0
```

---

In [1]:
""" DOESN't WORK as of now
Load kafka-spark driver to consume kafka messages.

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = ""
"""

""" IMPORTANT
# Load kafka spark driver, to receive kafka stream messages
# https://github.com/nodesense/cts-aws-spark-april-2021/blob/main/spark-setup-for-packages.md

# $KAFKA_HOME/bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic test
# $KAFKA_HOME/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic test
"""

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

ss = (
    SparkSession.builder.master("local[*]")
    .appName("sparkSTREAM:fromKAFKA")
    .getOrCreate()
)

In [ ]:
# # read from kafka, here spark is consumer for kafka topic called test
# # spark streaming works as dataframe/sql
# kafkaDf = spark.readStream.format("kafka")\
#   .option("kafka.bootstrap.servers", "192.168.93.128:9092")\
#   .option("subscribe", "test")\
#   .load()

In [ ]:
# # .show/print will not work directily due to stream..
# # linesDf.show() # worn't work
# kafkaDf.printSchema() # works

# # key is kafka key, in binary format
# # value is kafka value, in binary format
# # topic string
# # parition, integer
# # offer long 
# # timestamp - longint in ms
# # timestampType - Source Time, Record write time

In [ ]:

# # now convert kafka value which is in bytes to STRING, we ignore the key for now...
# # now we pick only value from the stream..
# linesDf = kafkaDf.selectExpr("CAST(value AS STRING)")
# linesDf.printSchema() # we get only value as string

In [ ]:
# # split the lines into words, then convert the words into individual row using a function called explode
# # explode will convert columns/array elements into spark record
# import pyspark.sql.functions as F
# # linesDf.value is a column
# # split convert to list of words [welcome, to, spark]
# # convert list of words into individual word/record
# # explode, will convert elements into record
# #wordsDf = linesDf.select(F.split(linesDf.value," "))
# # after explode the output would be, column name is shown as col
# #        welcome
# #         to
# #         spark
# # wordsDf = linesDf.select(F.explode(F.split(linesDf.value," ")) )

# wordsDf = linesDf.select(F.explode(F.split(linesDf.value," ")).alias("word") )
# #        welcome
# #         to
# #         spark
# # now the same result with col name word

# # generate running word count from stream
# # "word" is a column name
# wordCountsDf = wordsDf.groupBy("word").count()

In [ ]:

# # to print the data on console..
# # read the data send by nc command from linux terminal, print it on Jupyter console
# echoOnconsole = wordCountsDf\
#                 .writeStream\
#                 .outputMode("complete")\
#                 .format("console")\
#                 .start() # start the query. spark will subscribe for data

In [4]:
"""
SETUP: ensure Kafka Zookeeper/Broker/Producer are running

Read from Kafka.
Here, spark is the Consumer for kafka topic 'test'.
Spark streaming works as DataFrame/SQL
"""

topic = "test"

df_kafka = (
    ss.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "192.168.93.128:9092")
    .option("subscribe", topic)
    .load()
)
# df_lines = (
#     ss.readStream
#     .format("socket")
#     .option("host", "192.168.93.128")
#     .option("port", 9999)
#     .load()
# )

In [5]:
"""
.show()/print won't work as it's streaming data (not batch data)
.printSchema() works
"""

df_kafka.printSchema()

"""
# key: kafka key, in binary format
# value: kafka value, in binary format
# topic: string
# parition: integer
# offset: long 
# timestamp: longint in ms
# timestampType: Source Time, Record write time
"""

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



'\n# key is kafka key, in binary format\n# value is kafka value, in binary format\n# topic string\n# parition, integer\n# offer long \n# timestamp - longint in ms\n# timestampType - Source Time, Record write time\n'

In [6]:
"""
Convert kafka value (arrives as bytes) -> STRING; ignore the key for now..
Interested only in 'value' from the stream..
"""
df_lines = df_kafka.selectExpr("CAST(value AS STRING)")
df_lines.printSchema()

root
 |-- value: string (nullable = true)



In [7]:
"""
Split lines into words.
Convert words into individual spark records using explode (after splitting on the delimiter " ")
.explode() will convert invidual words into separate spark records
"""
import pyspark.sql.functions as f

df_words = df_lines.select(f.explode(f.split(df_lines.value, " ")).alias("word"))


In [8]:
df_word_count = df_words.groupBy("word").count()

In [9]:
"""
PRINT schema on console.
"""

echoOnConsole = (
    df_word_count
    .writeStream
    .outputMode("complete")  # options: append/complete
    .format("console")
    .start()  # Starts the query, and spark will subscribe to stream data
)

"""CHECK JUPYTER CONSOLE"""

In [ ]:
"""
FINALLY: Await termination (before exiting this notebook)
"""

# echoOnConsole.awaitTermination()

---

In [10]:
import uuid

In [11]:
help(uuid)

Help on module uuid:

NAME
    uuid - UUID objects (universally unique identifiers) according to RFC 4122.

MODULE REFERENCE
    https://docs.python.org/3.7/library/uuid
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    This module provides immutable UUID objects (class UUID) and the functions
    uuid1(), uuid3(), uuid4(), uuid5() for generating version 1, 3, 4, and 5
    UUIDs as specified in RFC 4122.
    
    If all you want is a unique ID, you should probably call uuid1() or uuid4().
    Note that uuid1() may compromise privacy since it creates a UUID containing
    the computer's network address.  uuid4() creates a random UUID.
    
    Typical usage:
    
        >>> import uuid
    
        